In [80]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 
from mlxtend.preprocessing import TransactionEncoder
import scipy as sp
from scipy import stats

raw_data_path = "/mnt/d/文档/grd/course/数据挖掘/assignments/04/wine-reviews/winemag-data_first150k.csv"
raw_data = pd.read_csv(raw_data_path)
raw_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [81]:
raw_data.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [82]:
#简单修复数据
#首先发现第一列没有表头名称
raw_data = raw_data.rename(columns={"Unnamed: 0": "index"})
raw_data.columns

Index(['index', 'country', 'description', 'designation', 'points', 'price',
       'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [83]:
#去掉城市，省份, 设计为空的行
raw_data.dropna(axis=0,subset=["country","province","designation"],inplace=True)
#去掉region_1,region_2同时为空的
raw_data.dropna(axis=0,how="all",subset=["region_1","region_2"],inplace=True)
#对region_2中为空的，用region_1的来填充
reg2_na_index = pd.isna(raw_data.loc[:,"region_2"])
raw_data.loc[reg2_na_index,"region_2"] = raw_data.loc[reg2_na_index,"region_1"]
# any(pd.isna(raw_data.loc[:,"region_2"]))

#对price中为空的，用众数来填充
price_nd = raw_data.loc[:,"price"].to_numpy(np.float32)
mode, count = stats.mode(price_nd,nan_policy="omit")
price_na_index = pd.isna(raw_data.loc[:,"price"])
raw_data.loc[price_na_index,"price"] = mode[0]

nan_l = []
for column in raw_data.columns:
    if any(pd.isna(raw_data.loc[:,column])):
        nan_l.append(column)
print(nan_l)

[]


In [84]:
#根据country, points, price来分析
sub_data = raw_data.loc[:,["country","points","price"]]
sub_data.head()

,country,points,price
0,US,96,235.0
1,Spain,96,110.0
2,US,96,90.0
3,US,96,65.0
4,France,95,66.0


In [85]:
#数据处理
tmp = sub_data.to_numpy(str)
X = tmp.tolist()
print(X[:10])

[['US', '96', '235.0'], ['Spain', '96', '110.0'], ['US', '96', '90.0'], ['US', '96', '65.0'], ['France', '95', '66.0'], ['Spain', '95', '73.0'], ['Spain', '95', '65.0'], ['Spain', '95', '110.0'], ['US', '95', '65.0'], ['US', '95', '60.0']]


In [86]:
#编码
te = TransactionEncoder()
te_ary = te.fit(X).transform(X)
encode_df = pd.DataFrame(te_ary, columns=te.columns_)
encode_df.head()

,10.0,100,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,...,98.0,99,99.0,Argentina,Australia,Canada,France,Italy,Spain,US
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [87]:
#使用apriori

freq_items = apriori(encode_df,min_support=0.05,use_colnames=True)
freq_items.head()

,support,itemsets
0,0.144089,(20.0)
1,0.058320,(84)
2,0.068984,(85)
3,0.091574,(86)
4,0.132221,(87)


In [88]:
#关联规则
rules = association_rules(freq_items, metric ="lift", min_threshold = 0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(France),(20.0),0.169306,0.144089,0.058472,0.345360,2.396862,0.034077,1.307455
1,(20.0),(France),0.144089,0.169306,0.058472,0.405804,2.396862,0.034077,1.398013
2,(US),(87),0.470145,0.132221,0.058904,0.125289,0.947569,-0.003259,0.992074
3,(87),(US),0.132221,0.470145,0.058904,0.445495,0.947569,-0.003259,0.955545
4,(90),(US),0.111279,0.470145,0.051475,0.462580,0.983909,-0.000842,0.985923
5,(US),(90),0.470145,0.111279,0.051475,0.109488,0.983909,-0.000842,0.997989


In [89]:
rules = association_rules(freq_items, metric="leverage",min_threshold=0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(France),(20.0),0.169306,0.144089,0.058472,0.345360,2.396862,0.034077,1.307455
1,(20.0),(France),0.144089,0.169306,0.058472,0.405804,2.396862,0.034077,1.398013


In [90]:
rules = association_rules(freq_items, metric="conviction",min_threshold=0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(France),(20.0),0.169306,0.144089,0.058472,0.345360,2.396862,0.034077,1.307455
1,(20.0),(France),0.144089,0.169306,0.058472,0.405804,2.396862,0.034077,1.398013
2,(US),(87),0.470145,0.132221,0.058904,0.125289,0.947569,-0.003259,0.992074
3,(87),(US),0.132221,0.470145,0.058904,0.445495,0.947569,-0.003259,0.955545
4,(90),(US),0.111279,0.470145,0.051475,0.462580,0.983909,-0.000842,0.985923
5,(US),(90),0.470145,0.111279,0.051475,0.109488,0.983909,-0.000842,0.997989
